## Closed-loop batch, constrained BO in BoTorch with qEI and qNEI

In this tutorial, we illustrate how to implement a simple Bayesian Optimization (BO) closed loop in BoTorch.

In general, we recommend for a relatively simple setup (like this one) to use Ax, since this will simplify your setup (including the amount of code you need to write) considerably. See the [Using BoTorch with Ax](./custom_botorch_model_in_ax) tutorial.

However, you may want to do things that are not easily supported in Ax at this time (like running high-dimensional BO using a VAE+GP model that you jointly train on high-dimensional input data). If you find yourself in such a situation, you will need to write your own optimization loop, as we do in this tutorial.


We use the batch Expected Improvement (qEI) and batch Noisy Expected Improvement (qNEI) acquisition functions to optimize a constrained version of the synthetic Hartmann6 test function. The standard problem is

$$f(x) = -\sum_{i=1}^4 \alpha_i \exp \left( -\sum_{j=1}^6 A_{ij} (x_j - P_{ij})^2  \right)$$

over $x \in [0,1]^6$ (parameter values can be found in `botorch/test_functions/hartmann6.py`).

In real BO applications, the design $x$ can influence multiple metrics in unknown ways, and the decision-maker often wants to optimize one metric without sacrificing another. To illustrate this, we add a synthetic constraint fo the form $\|x\|_1 - 3 \le 0$. Both the objective and the constraint are observed with noise. 

Since botorch assumes a maximization problem, we will attempt to maximize $-f(x)$ to achieve $\max_{x} -f(x) = 3.32237$.

In [1]:
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement, qNoisyExpectedImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
import time

import warnings
# warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
# warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore")



In [2]:
import torch
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

### Problem setup

First, we define the constraint used in the example in `outcome_constraint`. The second function `weighted_obj` is a "feasibility-weighted objective," which returns zero when not feasible. 

In [3]:
from botorch.test_functions.synthetic import Hartmann

neg_hartmann6 = Hartmann(negate=True)

In [4]:
from contextlib import contextmanager
# Helper functions
@contextmanager
def timer(label):
    import time
    start = time.time()
    yield
    print(f"[Process {label}] elasped in {time.time()-start}")
    

In [5]:
def outcome_constraint(X):
    """L1 constraint; feasible if less than or equal to zero."""
    return X.sum(dim=-1) - 3

def weighted_obj(X):
    """Feasibility weighted objective; zero if not feasible."""
    return neg_hartmann6(X) * (outcome_constraint(X) <= 0).type_as(X)

#### Model initialization

We use a `MultiOutputGP` to model the objective (output 0) and the constraint (output 1). We assume known homoskedastic observation noise on both the objective and constraint with standard error $\sigma = 0.5$. 

Each component is a `FixedNoiseGP`. The models are initialized with 10 points drawn randomly from $[0,1]^6$.

In [6]:
from botorch.models import FixedNoiseGP, ModelListGP, HeteroskedasticSingleTaskGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood

NOISE_SE = 0.5
NOISE_OF_NOISE = 0.1
NOISE_FURTHER = 0.2


def generate_initial_data(n=10):
    train_yvar = []
    # generate training data
    train_x = torch.rand(10, 6, device=device, dtype=dtype)
    exact_obj = neg_hartmann6(train_x).unsqueeze(-1)  # add output dimension
    exact_con = outcome_constraint(train_x).unsqueeze(-1)  # add output dimension
    observed_noise = NOISE_SE * torch.randn_like(exact_obj) + NOISE_OF_NOISE * torch.randn_like(exact_obj)
    train_obj = exact_obj + observed_noise + NOISE_FURTHER * torch.randn_like(exact_obj)
    train_con = exact_con + NOISE_SE * torch.randn_like(exact_con)
    best_observed_value = weighted_obj(train_x).max().item()
    train_yvar.append(observed_noise)
    return train_x, train_obj, train_con, best_observed_value, train_yvar
    
    
def initialize_model(train_x, train_obj, train_con, train_yvar, state_dict=None):
    train_yvar_con = torch.tensor(NOISE_SE**2, device=device, dtype=dtype)
    train_yvar_obj = torch.cat(train_yvar) ** 2
    # define models for objective and constraint
    model_obj = HeteroskedasticSingleTaskGP(train_x, train_obj, train_yvar_obj).to(train_x)
    model_con = HeteroskedasticSingleTaskGP(train_x, train_con, train_yvar_con.expand_as(train_con)).to(train_x)
    # combine into a multi-output GP model
    model = ModelListGP(model_obj, model_con)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    # load state dict if it is passed
    if state_dict is not None:
        model.load_state_dict(state_dict)
    return mll, model

#### Define a construct to extract the objective and constraint from the GP
The methods below take the outputs of the GP and return the objective and the constraint. In general, these can be any `Callable`, but here we simply need to index the correct output.

In [7]:
from botorch.acquisition.objective import ConstrainedMCObjective

def obj_callable(Z):
    return Z[..., 0]

def constraint_callable(Z):
    return Z[..., 1]

# define a feasibility-weighted objective for optimization
constrained_obj = ConstrainedMCObjective(
    objective=obj_callable,
    constraints=[constraint_callable],
)

#### Define a helper function that performs the essential BO step
The helper function below takes an acquisition function as an argument, optimizes it, and returns the batch $\{x_1, x_2, \ldots x_q\}$ along with the observed function values. For this example, we'll use a small batch of $q=3$. The function `optimize_acqf` optimizes the $q$ points jointly. A simple initialization heuristic is used to select the 10 restart initial locations from a set of 50 random points. 

In [8]:
from botorch.optim import optimize_acqf

BATCH_SIZE = 3
bounds = torch.tensor([[0.0] * 6, [1.0] * 6], device=device, dtype=dtype)


def optimize_acqf_and_get_observation(acq_func):
    """Optimizes the acquisition function, and returns a new candidate and a noisy observation."""
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=BATCH_SIZE,
        num_restarts=10,
        raw_samples=500,  # Sample on GP using Sobel sequence
        options={
            "batch_limit": 5,
            "max_iter": 200,
            "seed": 0
        }
    )
    train_yvar = []
    # observe new values <-- new values -> obtained in batches?
    new_x = candidates.detach()
    exact_obj = neg_hartmann6(new_x).unsqueeze(-1)  # add output dimension
    exact_con = outcome_constraint(new_x).unsqueeze(-1)  # add output dimension
    observed_noise = NOISE_SE * torch.randn_like(exact_obj) + NOISE_OF_NOISE * torch.randn_like(exact_obj)
    new_obj = exact_obj + observed_noise + NOISE_FURTHER * torch.randn_like(exact_obj)
    new_con = exact_con + NOISE_SE * torch.randn_like(exact_con)
    train_yvar.append(observed_noise)
    return new_x, new_obj, new_con, train_yvar


def update_random_observations(best_random):
    """Simulates a random policy by taking a the current list of best values observed randomly,
    drawing a new random point, observing its value, and updating the list.
    """
    rand_x = torch.rand(BATCH_SIZE, 6)
    next_random_best = weighted_obj(rand_x).max().item()
    best_random.append(max(best_random[-1], next_random_best))       
    return best_random

### Perform Bayesian Optimization loop with qNEI
The Bayesian optimization "loop" for a batch size of $q$ simply iterates the following steps:
1. given a surrogate model, choose a batch of points $\{x_1, x_2, \ldots x_q\}$
2. observe $f(x)$ for each $x$ in the batch 
3. update the surrogate model. 


Just for illustration purposes, we run three trials each of which do `N_BATCH=20` rounds of optimization. The acquisition function is approximated using `MC_SAMPLES=500` samples.

*Note*: Running this may take a little while.

from copy import copy
N_TRIALS = 20
N_BATCH = 20
MC_SAMPLES = 500

verbose = True

best_observed_all_ei, best_observed_all_nei, best_random_all = [], [], []

# average over multiple trials
for trial in range(1, N_TRIALS + 1):
    
    print(f"\nTrial {trial:>2} of {N_TRIALS} ", end="")
    best_observed_ei, best_observed_nei, best_random = [], [], []
    
    # call helper functions to generate initial training data and initialize model
    train_x_ei, train_obj_ei, train_con_ei, best_observed_value_ei, train_yvar_ei = generate_initial_data(n=10)
    mll_ei, model_ei = initialize_model(train_x_ei, train_obj_ei, train_con_ei, train_yvar_ei)
    
    train_x_nei, train_obj_nei, train_con_nei, train_yvar_nei = train_x_ei, train_obj_ei, train_con_ei, copy(train_yvar_ei)
    best_observed_value_nei = best_observed_value_ei
    mll_nei, model_nei = initialize_model(train_x_nei, train_obj_nei, train_con_nei, train_yvar_nei)
    
    best_observed_ei.append(best_observed_value_ei)
    best_observed_nei.append(best_observed_value_nei)
    best_random.append(best_observed_value_ei)
    
    # run N_BATCH rounds of BayesOpt after the initial random batch
    for iteration in range(1, N_BATCH + 1):    
        
        t0 = time.time()
        
        # fit the models
        fit_gpytorch_model(mll_ei)
        fit_gpytorch_model(mll_nei)
        
        # define the qEI and qNEI acquisition modules using a QMC sampler
        qmc_sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES, seed=0)
        
        # for best_f, we use the best observed noisy values as an approximation
        qEI = qExpectedImprovement(
            model=model_ei, 
            best_f=(train_obj_ei * (train_con_ei <= 0).to(train_obj_ei)).max(),
            sampler=qmc_sampler, 
            objective=constrained_obj,
        )
        
        qNEI = qNoisyExpectedImprovement(
            model=model_nei, 
            X_baseline=train_x_nei,
            sampler=qmc_sampler, 
            objective=constrained_obj,
        )
        
        # optimize and get new observation
        new_x_ei, new_obj_ei, new_con_ei, new_yvar_ei = optimize_acqf_and_get_observation(qEI)
        new_x_nei, new_obj_nei, new_con_nei, new_yvar_nei = optimize_acqf_and_get_observation(qNEI)
                
        # update training points
        train_x_ei = torch.cat([train_x_ei, new_x_ei])
        train_obj_ei = torch.cat([train_obj_ei, new_obj_ei])
        train_con_ei = torch.cat([train_con_ei, new_con_ei])
        train_yvar_ei.extend(new_yvar_ei)

        train_x_nei = torch.cat([train_x_nei, new_x_nei])
        train_obj_nei = torch.cat([train_obj_nei, new_obj_nei])
        train_con_nei = torch.cat([train_con_nei, new_con_nei])
        train_yvar_nei.extend(new_yvar_nei)

        # update progress
        best_random = update_random_observations(best_random)
        best_value_ei = weighted_obj(train_x_ei).max().item()
        best_value_nei = weighted_obj(train_x_nei).max().item()
        best_observed_ei.append(best_value_ei)
        best_observed_nei.append(best_value_nei)

        # reinitialize the models so they are ready for fitting on next iteration
        # use the current state dict to speed up fitting
        mll_ei, model_ei = initialize_model(
            train_x_ei, 
            train_obj_ei, 
            train_con_ei,
            train_yvar_ei,
            model_ei.state_dict(),
        )
        mll_nei, model_nei = initialize_model(
            train_x_nei, 
            train_obj_nei, 
            train_con_nei,
            train_yvar_nei,
            model_nei.state_dict(),
        )
        
        t1 = time.time()
        
        if verbose:
            print(
                f"\nBatch {iteration:>2}: best_value (random, qEI, qNEI) = "
                f"({max(best_random):>4.2f}, {best_value_ei:>4.2f}, {best_value_nei:>4.2f}), "
                f"time = {t1-t0:>4.2f}.", end=""
            )
        else:
            print(".", end="")
   
    best_observed_all_ei.append(best_observed_ei)
    best_observed_all_nei.append(best_observed_nei)
    best_random_all.append(best_random)

In [9]:
from collections import OrderedDict
train_x_ei = torch.tensor([[0.5176, 0.2980, 0.6463, 0.2611, 0.5728, 0.2798],
         [0.0277, 0.6281, 0.6837, 0.1972, 0.5204, 0.6244],
         [0.9555, 0.0461, 0.1888, 0.9081, 0.1599, 0.0548],
         [0.6722, 0.0966, 0.5144, 0.3461, 0.8859, 0.3065],
         [0.9386, 0.4626, 0.3744, 0.8504, 0.1106, 0.1222],
         [0.8524, 0.3512, 0.8365, 0.1308, 0.1649, 0.6388],
         [0.5696, 0.8921, 0.5743, 0.5117, 0.9085, 0.3676],
         [0.6295, 0.2520, 0.7023, 0.2467, 0.3688, 0.4187],
         [0.2932, 0.1394, 0.1999, 0.1387, 0.9179, 0.6922],
         [0.1211, 0.7584, 0.1315, 0.8753, 0.6155, 0.2026],
         [0.4284, 0.3876, 0.6961, 0.1948, 0.3626, 0.5070],
         [0.7016, 0.1445, 0.6422, 0.2589, 0.2991, 0.3089],
         [0.5052, 0.1628, 0.7159, 0.3714, 0.3281, 0.4779],
         [0.3262, 0.5099, 0.8146, 0.1784, 0.3715, 0.4607],
         [0.4123, 0.5344, 0.5985, 0.2132, 0.3729, 0.5470],
         [0.3749, 0.3599, 0.6758, 0.0833, 0.4001, 0.6054],
         [0.1684, 0.3411, 0.6374, 0.1022, 0.3959, 0.6607],
         [0.4763, 0.3824, 0.7066, 0.1453, 0.4228, 0.6945],
         [0.3750, 0.3125, 0.5531, 0.1303, 0.3832, 0.5398],
         [0.3132, 0.4162, 0.6855, 0.1669, 0.3752, 0.7157],
         [0.2366, 0.3256, 0.7042, 0.1616, 0.4207, 0.7017],
         [0.1786, 0.6945, 0.2689, 0.0858, 0.3230, 0.4821],
         [0.3290, 0.2128, 0.6420, 0.1755, 0.6976, 0.3593],
         [0.2788, 0.3243, 0.6790, 0.1616, 0.4301, 0.5848],
         [0.3087, 0.4728, 0.6703, 0.0936, 0.3552, 0.8081],
         [0.4974, 0.3587, 0.7830, 0.0056, 0.3744, 0.6762],
         [0.8183, 0.9164, 0.6991, 0.8839, 1.0000, 0.8439],
         [0.4545, 0.4221, 0.6846, 0.3519, 0.3925, 0.7313]], device='cuda:0',
        dtype=torch.float64)
train_obj_ei = torch.tensor([[-0.2401],
         [ 1.3177],
         [ 0.6125],
         [-0.7935],
         [ 0.3450],
         [-0.0801],
         [ 0.8322],
         [ 1.3013],
         [-0.0137],
         [-0.8754],
         [ 2.1543],
         [ 0.5921],
         [ 0.5986],
         [ 1.1937],
         [ 1.4724],
         [ 2.0907],
         [ 2.3193],
         [ 2.4420],
         [ 1.1794],
         [ 2.8481],
         [ 1.7328],
         [ 0.2378],
         [-0.0942],
         [ 0.9635],
         [ 1.1507],
         [ 0.8870],
         [-0.3474],
         [ 2.8230]], device='cuda:0', dtype=torch.float64)
train_con_ei = torch.tensor([[-1.1204],
         [-0.0179],
         [-1.5471],
         [-0.5860],
         [-0.5636],
         [ 0.3776],
         [ 1.3285],
         [-0.4716],
         [-0.6478],
         [ 0.0865],
         [-1.6471],
         [-0.1833],
         [-0.3754],
         [-0.4154],
         [-0.7351],
         [ 0.4229],
         [-0.4502],
         [-0.1022],
         [-0.5263],
         [-0.7971],
         [-0.9095],
         [-1.3557],
         [ 0.0699],
         [-0.3544],
         [-0.9900],
         [ 0.2022],
         [ 1.6931],
         [ 1.0082]], device='cuda:0', dtype=torch.float64)
train_yvar_ei = [torch.tensor([[-0.6089],
          [ 0.5122],
          [ 0.4256],
          [-0.7424],
          [ 0.2856],
          [-0.3044],
          [ 0.4848],
          [ 0.2162],
          [ 0.1593],
          [-1.0060]], device='cuda:0', dtype=torch.float64), torch.tensor([[ 0.5837],
          [-0.0038],
          [-0.8226]], device='cuda:0', dtype=torch.float64), torch.tensor([[0.1648],
          [0.3299],
          [0.8004]], device='cuda:0', dtype=torch.float64), torch.tensor([[ 0.3482],
          [ 0.8111],
          [-0.7657]], device='cuda:0', dtype=torch.float64), torch.tensor([[ 0.9292],
          [-0.0358],
          [-0.3752]], device='cuda:0', dtype=torch.float64), torch.tensor([[-0.8108],
          [-1.1008],
          [-0.4832]], device='cuda:0', dtype=torch.float64), torch.tensor([[-0.0025],
          [-0.2966],
          [ 1.1818]], device='cuda:0', dtype=torch.float64)]
best_value_ei = weighted_obj(train_x_ei).max().item()
state_dict = OrderedDict([('models.0.likelihood.noise_covar.noise_model.likelihood.noise_covar.raw_noise',
              torch.tensor([0.0467], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.a',
              torch.tensor([-3.], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.b',
              torch.tensor([5.], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.sigma',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.loc',
              torch.tensor([0.], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.scale',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.mean_module.constant',
              torch.tensor([-4.3501], device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.covar_module.raw_outputscale',
              torch.tensor(12.2839, device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.covar_module.base_kernel.raw_lengthscale',
              torch.tensor([[-0.8111, -1.4067, -0.9732, -1.4227, -1.0661, -1.2290]],
                     device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.concentration',
              torch.tensor(3., device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.rate',
              torch.tensor(6., device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.covar_module.outputscale_prior.concentration',
              torch.tensor(2., device='cuda:0', dtype=torch.float64)),
             ('models.0.likelihood.noise_covar.noise_model.covar_module.outputscale_prior.rate',
              torch.tensor(0.1500, device='cuda:0', dtype=torch.float64)),
             ('models.0.mean_module.constant',
              torch.tensor([0.2254], device='cuda:0', dtype=torch.float64)),
             ('models.0.covar_module.raw_outputscale',
              torch.tensor(-0.6153, device='cuda:0', dtype=torch.float64)),
             ('models.0.covar_module.base_kernel.raw_lengthscale',
              torch.tensor([[-0.2304, -0.8648, -1.5052, -0.9083, -1.5879, -0.8495]],
                     device='cuda:0', dtype=torch.float64)),
             ('models.0.covar_module.base_kernel.lengthscale_prior.concentration',
              torch.tensor(3., device='cuda:0', dtype=torch.float64)),
             ('models.0.covar_module.base_kernel.lengthscale_prior.rate',
              torch.tensor(6., device='cuda:0', dtype=torch.float64)),
             ('models.0.covar_module.outputscale_prior.concentration',
              torch.tensor(2., device='cuda:0', dtype=torch.float64)),
             ('models.0.covar_module.outputscale_prior.rate',
              torch.tensor(0.1500, device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.likelihood.noise_covar.raw_noise',
              torch.tensor([0.0494], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.a',
              torch.tensor([-3.], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.b',
              torch.tensor([5.], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.sigma',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.loc',
              torch.tensor([0.], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.scale',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.mean_module.constant',
              torch.tensor([-1.3694], device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.covar_module.raw_outputscale',
              torch.tensor(3.6496, device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.covar_module.base_kernel.raw_lengthscale',
              torch.tensor([[-0.8514, -0.8702, -0.8826, -0.8748, -0.8927, -0.8636]],
                     device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.concentration',
              torch.tensor(3., device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.rate',
              torch.tensor(6., device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.covar_module.outputscale_prior.concentration',
              torch.tensor(2., device='cuda:0', dtype=torch.float64)),
             ('models.1.likelihood.noise_covar.noise_model.covar_module.outputscale_prior.rate',
              torch.tensor(0.1500, device='cuda:0', dtype=torch.float64)),
             ('models.1.mean_module.constant',
              torch.tensor([-0.0006], device='cuda:0', dtype=torch.float64)),
             ('models.1.covar_module.raw_outputscale',
              torch.tensor(-0.0489, device='cuda:0', dtype=torch.float64)),
             ('models.1.covar_module.base_kernel.raw_lengthscale',
              torch.tensor([[-0.7549, -0.7048, -0.6754, -1.7126, -0.6973, -0.7827]],
                     device='cuda:0', dtype=torch.float64)),
             ('models.1.covar_module.base_kernel.lengthscale_prior.concentration',
              torch.tensor(3., device='cuda:0', dtype=torch.float64)),
             ('models.1.covar_module.base_kernel.lengthscale_prior.rate',
              torch.tensor(6., device='cuda:0', dtype=torch.float64)),
             ('models.1.covar_module.outputscale_prior.concentration',
              torch.tensor(2., device='cuda:0', dtype=torch.float64)),
             ('models.1.covar_module.outputscale_prior.rate',
              torch.tensor(0.1500, device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.likelihood.noise_covar.raw_noise',
              torch.tensor([0.0467], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.likelihood.noise_covar.noise_prior.a',
              torch.tensor([-3.], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.likelihood.noise_covar.noise_prior.b',
              torch.tensor([5.], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.likelihood.noise_covar.noise_prior.sigma',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.loc',
              torch.tensor([0.], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.scale',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.mean_module.constant',
              torch.tensor([-4.3501], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.covar_module.raw_outputscale',
              torch.tensor(12.2839, device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.covar_module.base_kernel.raw_lengthscale',
              torch.tensor([[-0.8111, -1.4067, -0.9732, -1.4227, -1.0661, -1.2290]],
                     device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.concentration',
              torch.tensor(3., device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.rate',
              torch.tensor(6., device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.covar_module.outputscale_prior.concentration',
              torch.tensor(2., device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.0.noise_covar.noise_model.covar_module.outputscale_prior.rate',
              torch.tensor(0.1500, device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.likelihood.noise_covar.raw_noise',
              torch.tensor([0.0494], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.likelihood.noise_covar.noise_prior.a',
              torch.tensor([-3.], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.likelihood.noise_covar.noise_prior.b',
              torch.tensor([5.], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.likelihood.noise_covar.noise_prior.sigma',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.loc',
              torch.tensor([0.], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.likelihood.noise_covar.noise_prior.tails.scale',
              torch.tensor([0.5000], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.mean_module.constant',
              torch.tensor([-1.3694], device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.covar_module.raw_outputscale',
              torch.tensor(3.6496, device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.covar_module.base_kernel.raw_lengthscale',
              torch.tensor([[-0.8514, -0.8702, -0.8826, -0.8748, -0.8927, -0.8636]],
                     device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.concentration',
              torch.tensor(3., device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.covar_module.base_kernel.lengthscale_prior.rate',
              torch.tensor(6., device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.covar_module.outputscale_prior.concentration',
              torch.tensor(2., device='cuda:0', dtype=torch.float64)),
             ('likelihood.likelihoods.1.noise_covar.noise_model.covar_module.outputscale_prior.rate',
              torch.tensor(0.1500, device='cuda:0', dtype=torch.float64))])

In [ ]:
for seed in range(500, 20000):
    print(f"Seed: {seed}")
    torch.manual_seed(seed)
    mll_ei, model_ei = initialize_model(
        train_x_ei, 
        train_obj_ei, 
        train_con_ei,
        train_yvar_ei,
        state_dict
    )
    fit_gpytorch_model(mll_ei)
    qmc_sampler = SobolQMCNormalSampler(num_samples=500, seed=seed)
    qEI = qExpectedImprovement(
        model=model_ei, 
        best_f=(train_obj_ei * (train_con_ei <= 0).to(train_obj_ei)).max(),
        sampler=qmc_sampler, 
        objective=constrained_obj,
    )
    optimize_acqf_and_get_observation(qEI)

Seed: 500
Seed: 501
Seed: 502
Seed: 503
Seed: 504
Seed: 505
Seed: 506
Seed: 507
Seed: 508
Seed: 509
Seed: 510
Seed: 511
Seed: 512
Seed: 513
Seed: 514
Seed: 515
Seed: 516
Seed: 517
Seed: 518
Seed: 519
Seed: 520
Seed: 521
Seed: 522
Seed: 523
Seed: 524
Seed: 525
Seed: 526
Seed: 527
Seed: 528
Seed: 529
Seed: 530
Seed: 531
Seed: 532
Seed: 533
Seed: 534
Seed: 535
Seed: 536
Seed: 537
Seed: 538
Seed: 539
Seed: 540
Seed: 541
Seed: 542
Seed: 543
Seed: 544
Seed: 545
Seed: 546
Seed: 547
Seed: 548
Seed: 549
Seed: 550
Seed: 551
Seed: 552
Seed: 553
Seed: 554
Seed: 555
Seed: 556
Seed: 557
Seed: 558
Seed: 559
Seed: 560
Seed: 561
Seed: 562
Seed: 563
Seed: 564
Seed: 565
Seed: 566
Seed: 567
Seed: 568
Seed: 569
Seed: 570
Seed: 571
Seed: 572
Seed: 573
Seed: 574
Seed: 575
Seed: 576
Seed: 577
Seed: 578
Seed: 579
Seed: 580
Seed: 581
Seed: 582
Seed: 583
Seed: 584
Seed: 585
Seed: 586
Seed: 587
Seed: 588
Seed: 589
Seed: 590
Seed: 591
Seed: 592
Seed: 593
Seed: 594
Seed: 595
Seed: 596
Seed: 597
Seed: 598
Seed: 599


#### Plot the results
The plot below shows the best objective value observed at each step of the optimization for each of the algorithms. The confidence intervals represent the variance at that step in the optimization across the trial runs.

import numpy as np

GLOBAL_MAXIMUM = neg_hartmann6.optimal_value

from matplotlib import pyplot as plt
%matplotlib widget

def ci(y):
    return 1.96 * y.std(axis=0) / np.sqrt(N_TRIALS)

iters = np.arange(N_BATCH + 1) * BATCH_SIZE
y_ei = np.asarray(best_observed_all_ei)
y_nei = np.asarray(best_observed_all_nei)
y_rnd = np.asarray(best_random_all)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.errorbar(iters, y_rnd.mean(axis=0), yerr=ci(y_rnd), label="random", linewidth=1.5)
ax.errorbar(iters, y_ei.mean(axis=0), yerr=ci(y_ei), label="qEI", linewidth=1.5)
ax.errorbar(iters, y_nei.mean(axis=0), yerr=ci(y_nei), label="qNEI", linewidth=1.5)
plt.plot([0, N_BATCH * BATCH_SIZE], [GLOBAL_MAXIMUM] * 2, 'k', label="true best bjective", linewidth=2)
ax.set_ylim(bottom=0.5)
ax.set(xlabel='number of observations (beyond initial points)', ylabel='best objective value')
ax.legend(loc="lower right")

plt.show()